#### word2vec

In [2]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran  
import time

In [3]:
# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
    with open(filename, 'r', encoding="utf-8") as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]  # header 제거
    return data


# 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('data/ratings.txt')
print(len(review_data))  # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료:  0.45456957817077637


    ① 네이버 영화 리뷰 말뭉치 파일(ratings.txt)을 읽어와 리스트로 변환하는 함수입니다. 
    ratings.txt 파일은 라인마다 Tab 을 사용해서 id, document, label 컬럼으로 데이터가 구분되어있습니다. 
    따라서  read_review_data() 함수는 리뷰 데이터를 각 라인 별로 읽어와 \t를 기준으로 데이터를 분리합니다.  
    그 후 첫 번째 행의 헤더를 제거하고 리뷰 데이터만 반환합니다.

    ①에서 정의한 read_review_data()함수를 호출해 현재 경로에 있는 ratings.txt 파일을 리스트 형태로 읽어옵니다.


In [4]:
# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)

2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  163.98976731300354


    ④ Komoran 형태소 분석기를 이용해 불러온 리뷰 데이터에서 문장별로 명사만 추출합니다.  
    여기서 sentence[1]은 rating.txt 파일에서 document 컬럼의 데이터를 의미합니다.


In [5]:
start = time.time()
# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1)

print('3) word2vec 모델 학습 완료: ', time.time() - start)

3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료:  16.478259563446045


    ⑤ ④에서 추출한 명사 리스트로 Word2Vec모델을 학습시킵니다. 
    여기서 Word2Vec()의 주요 하이퍼파라미터는 다음과 같습니다.
    
    vector_size의 경우 10만개의 데이터의 경우 100개의 vector_size가 적절하다.
    window는 실험적으로 경험에 의한 값을 많이 사용한다.
    min_count의 경우에는 2를 대체로 사용한다.
    


In [6]:
start = time.time()
# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('data/nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)

4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  0.2729966640472412


    ⑥ 모델을 현재 디렉터리에 ‘nvmc.model’ 이란 이름으로 저장합니다.


In [7]:
# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

corpus_count :  200000
corpus_total_words :  1076896


    주요 실행 시점마다 계산 시간을 측정했으며, 
    전처리 과정으로 리뷰 데이터 20만 개에서 명사를 추출하는데 약 131초 정도 소요되었고, 
    약 100만개 정도의 단어 임베딩을 처리할 수 있는 Word2Vec 모델을 학습하고 저장하는 데 약 19초 정도 소요되었습니다.


##### 단어 임베딩된 값과 벡터 공간상의 유사한 단어들을 확인하는 예제

In [3]:
from gensim.models import Word2Vec

# 모델 로딩
model = Word2Vec.load('data/nvmc.model')  # ① 앞서 생성한 모델 파일을 읽어와 Word2Vec 객체를 생성합니다.
print("corpus_total_words : ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print('사랑 : ', model.wv['사랑'])  # 모델을 학습할 때 설정한 size 하이퍼파라미터만큼 단어 임베딩 벡터 차원 크기가 결정됩니다.



corpus_total_words :  1076896
사랑 :  [-0.09239019 -0.6113379   0.20210013  0.03807115 -0.20520669 -0.1487556
 -0.269031    0.13204677 -0.04312486  0.40844262 -0.13141038 -0.07152593
  0.09941676 -0.19877242 -0.15639223  0.5141167  -0.03041169 -0.37351817
 -0.21048227 -0.48086122  0.3776593   0.13476379  0.12408568  0.10856127
 -0.16274454  0.00927417  0.03441136 -0.11233966 -0.16343175 -0.16550887
  0.14632772  0.06695759  0.41403937 -0.08960392  0.14960782  0.20262213
  0.08305626 -0.07944804 -0.35352397 -0.2742586  -0.20479113  0.1189067
 -0.11226966 -0.33121994  0.45288533  0.24256226  0.03242411 -0.1332587
  0.4911213  -0.03512859  0.11811792 -0.14201482  0.03427875  0.02912429
 -0.24255635 -0.20937502  0.03123922 -0.28391117 -0.1790119   0.1610645
 -0.05075287  0.11262257 -0.29918575  0.16835694 -0.00819165  0.08042238
 -0.40971318  0.03663733 -0.11486392  0.62863564  0.21270254 -0.05054173
  0.3662721   0.18001905  0.2973186  -0.0118237   0.40479502 -0.16692252
 -0.22196138 -0.378

In [9]:
# 단어 유사도 계산
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))  
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))  
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))  
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))  
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))

# 가장 유사한 단어 추출
print(model.wv.most_similar("안성기", topn=5))  
print(model.wv.most_similar("시리즈", topn=5))


일요일 = 월요일	 0.92875725
안성기 = 배우	 0.715711
대기업 = 삼성	 0.8698282
일요일 != 삼성	 0.63510627
히어로 != 삼성	 0.42573237
[('정려원', 0.9387677311897278), ('정재영', 0.9337273240089417), ('재발견', 0.9329094290733337), ('설경구', 0.9327645301818848), ('지진희', 0.9311741590499878)]
[('엑스맨', 0.8029845952987671), ('포터', 0.7887096405029297), ('반지의 제왕', 0.7793396711349487), ('해리', 0.7742770314216614), ('에이리언', 0.7708970308303833)]


    Ⓒ gensim 패키지의 model.wv.most_similarity() 함수를 호출할 경우 인자로 사용한 단어와 가장 유사한 단어를 리스트로 반환해 줍니다. 
    즉, 벡터 공간에서 가장 가까운 거리에  있는 단어들을 반환합니다. 
    topn 인자는 반환되는 유사한 단어 수를 의미하며, 예제에서는 5개 까지 유사한 단어를반환합니다.


#### n-gram

In [ ]:
# 어절 단위 n-gram
def word_ngram(bow, num_gram):
    text = tuple(bow)
    ngrams = [text[x:x + num_gram] for x in range(0, len(text))]
    return tuple(ngrams)

    ① 어절 단위로 n-gram 토큰을 추출하는 함수입니다. 추출된 토큰들은 튜플 형태로 반환됩니다.
    이전 예시처럼 슬라이싱을 이용해 문장을 어절 단위로 n개씩 끊어서 토큰을 저장합니다.


In [ ]:
# 유사도 계산
def similarity(doc1, doc2):  
    cnt = 0
    for token in doc1:
        if token in doc2:  
            cnt = cnt + 1
            
    return cnt/len(doc1)


    Ⓒ doc1의 토큰이 doc2의 토큰과 얼마나 동일한지 횟수를 카운트 합니다. 
    카운트된 값을  doc1의 전체 토큰 수로 나누면 유사도가 계산됩니다. 
    이때 결과가 1.0에 가까울수록 doc1과 유사해 집니다.


In [ ]:
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학하였다'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학하였다'
sentence3 = '나는 맛있는 밥을 뉴턴 선생님과 함께 먹었습니다'

In [ ]:
#형태소 분석기에서 명사(단어) 추출
komoran = Komoran()
bow1 = komoran,nouns(sentences1)
bow2 = komoran,nouns(sentences2)
bow3 = komoran,nouns(sentences3)

In [ ]:
#단어 n-gram 토큰 추출
doc1 = word_ngram(bow1, 2) #2-gram 방식으로  추출
doc2 = word_ngram(bow2, 2)  
doc3 = word_ngram(bow3, 2)


    word_ngram() 함수를 이용해 정의된 문장에서 명사를 리스트 형태로 추출합니다.
    여기서 word_ngram() 함수의 num_gram 인자에 2를 입력했으므로 2-gram 방식으로 토큰을 추출합니다.


In [ ]:
#추출된 n-gram 토큰 출력  
print(doc1)  
print(doc2)  
print(doc3)


In [ ]:
#유사도 계산
r1 = similarity(doc1, doc2)  
r2 = similarity(doc3, doc1)


In [ ]:
#계산된 유사도 출력  
print(r1)  
print(r2)


#### 코사인 유사도

In [ ]:
from konlpy.tag import Komoran
import numpy as np
from numpy import dot
from numpy.linalg import norm


# 코사인 유사도 계산
def cos_sim(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

    ① 코사인 유사도를 계산하는 함수입니다. 
    코사인 유사도 계산에는 넘파이에서 제공하는 벡터 내적을 계산하는 함수와 노름(𝑛𝑜𝑟𝑚)을 계산하는 함수를 이용합니다.
    넘파이의 dot()함수는 인자로 들어온 2개의 넘파이 배열을 내적곱(𝑑𝑜𝑡 𝑝𝑟𝑜𝑑𝑢𝑐𝑡)합니다. 
    넘파이에서는 norm()함수를 제공합니다.
    여기서 노름은 벡터의 크기를 나타내는 수학 용어입니다. 
    벡터의 노름에는 여러가지 종류가 있지만 코사인 유사도에서는 L2 노름(유클리드 노름)을 주로 사용합니다. 
    L2 노름의 수식은 아래와 같습니다.
    
    코사인 유사도 수식의 분모에 있는 벡터의 크기를 계산하는 부분과 동일하기 때문에 벡터의 크기 계산에 넘파이 norm() 함수를 사용합니다.
    norm()함수의 인자로 들어온 넘파이 배열의 노름 크기를 계산합니다.



In [ ]:
# TDM 만들기
def make_term_doc_mat(sentence_bow, word_dics):  
    freq_mat = {}
    for word in word_dics:  
        freq_mat[word] = 0

    for word in word_dics:
        if word in sentence_bow:  
            freq_mat[word] += 1

    return freq_mat


    Ⓒ 비교 문장에서 추출한 단어 사전을 기준으로 문장에 해당 단어들이 얼마나 포함되어 있는지 나타내는 
    단어 문서 행렬 TDM을 만들어주는 함수 입니다.


In [ ]:
# 단어 벡터 만들기
def make_vector(tdm):
    vec = []
    for key in tdm:
        vec.append(tdm[key])
    return vec

    ③ 단어 문서 행렬에서 표현된 토큰들의 출현 빈도 데이터를 벡터로 만들어 주는 함수입니다.


In [ ]:
# 문장 정의
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학하였다'  
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학하였다'  
sentence3 = '나는 맛잇는 밥을 뉴턴 선생님과 함께 먹었습니다.'

# 헝태소분석기를 이용해 단어 묶음 리스트 생성
komoran = Komoran()
bow1 = komoran.nouns(sentence1)  
bow2 = komoran.nouns(sentence2)  
bow3 = komoran.nouns(sentence3)

# 단어 묶음 리스트를 하나로 합침
bow = bow1 + bow2 + bow3

# 하나로 합쳐진 단어 묶음 리스트에서 중복된 단어를 제거해 새로운 단어 사전 리스트를 구축
word_dics = []  
for token in bow:
    if token not in word_dics:
        word_dics.append(token)


In [ ]:
# 문장 별 단어 문서 행렬 계산
freq_list1 = make_term_doc_mat(bow1, word_dics)  
freq_list2 = make_term_doc_mat(bow2, word_dics)  
freq_list3 = make_term_doc_mat(bow3, word_dics)

# 각 문장마다 단어 문서 행렬 리스트를 만든 후 출력합니다.
print(freq_list1)  
print(freq_list2)  
print(freq_list3)


In [ ]:
# 각 문장마다 벡터를 생성해 넘파이 배열로 변환
doc1 = np.array(make_vector(freq_list1))  
doc2 = np.array(make_vector(freq_list2))  
doc3 = np.array(make_vector(freq_list3))

# 코사인 유사도 계산
r1 = cos_sim(doc1, doc2)  
r2 = cos_sim(doc3, doc1)  

print(r1)
print(r2)
